In [2]:
from collections import defaultdict
import os
import csv
import dateutil.parser
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
def reader(directory= "./data/"):
    listOfLists = []
    for f in os.listdir(directory):
        if f.endswith(".txt"):
            f = directory + f
            with open(f, "rb") as csvfile:
                
                #SKIP HEADER ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS     ']
                next(csvfile)
                for line in csvfile:
                    listOfLists.append(line.split(","))
    
    #Remove excess space and \n from last item
    for item in listOfLists[0:80000]:
        item[-1] = item[-1][0:10]
                    
                    

    return listOfLists[0:80000]
    print listOfLists


reader()

OSError: [Errno 2] No such file or directory: './data/'

**CHALLENGE 1**

In [ ]:
def ConvertRawDataToDict(inlist):
    d = defaultdict(list)
    #d = {}
    
    for row in inlist:
        key = tuple(row[0:4])
        vals = row[4:]

        if key in d:
            d[key].append(vals)
        else:
            valsRow = list(vals)
            d[key] = [valsRow]
            
    return d

d = ConvertRawDataToDict(reader())
print len(d)


**CHALLENGE 2**

In [ ]:
def timesDict(dictionary):
    
    newDict = defaultdict()
    
    for key, value in dictionary.items():
        newDict[key] = []
        
               
        for row in value: 
            #for each key, make date time into one string        
            dateStr = row[2] + " " + row[3]
            
            #use dateutil to convert date time
            daTime = dateutil.parser.parse(dateStr)
            
            
            #convert entries to integer to get rid of leading 0's
            entryInt = int(row[5])
            
            
            #creates list of lists with DaTime, Entry count
            newDataRow = []
            newDataRow.append(daTime)
            newDataRow.append(entryInt)
            
            newDict[key].append(newDataRow)
        
    for k, val in newDict.items():
        
        #sort date
        val = sorted(val, key = lambda x: x[0])
        
        #subtract previous count from recent count
        for i, row in enumerate(val):
            if i == 0:
                row.append(0)
            else:
                row.append(val[i][1] - val[i-1][1])
        
        #remove old count value
        for item in val:
            item.pop(-2)
    return newDict
        
    
d =  timesDict(d)

# for key,value in d.items()


**CHALLENGE 3**

In [ ]:
#n = 4, data comes every 4 hours
#OBJECTIVE: same keys, single count for single day

def perDay(d):
    bydayDict = defaultdict()
    
    for key, val in d.items():
        
        
        bydayDict[key] = []
        
        bydayDate = ""
        
        count = 0
        for index, ls in enumerate(sorted(val)):
            try:
                date = ls[0].date()
            except AttributeError:
                date = ls[0]
            entry = ls[1]
            
            if index == 0:
                lastDate = date
            if date == lastDate:
                count += entry
            else:
                bydayDict[key].append([lastDate,count])
                lastDate = date
                count = entry
        bydayDict[key].append([lastDate,count])
        
    return bydayDict
            
daily = perDay(d)


**CHALLENGE 4**

In [ ]:
#to choose  to print for one day
def PlotbyDay(dictionary, key):
    dates = [date[0] for date in dictionary[key]]
    counts = [count[1] for count in dictionary[key]]
    
    return dates, counts


In [ ]:
def TimeSeries(dates, counts):
    plt.figure(figsize=(10,5))
    plt.plot(dates, counts)
    
dates, counts = PlotbyDay(daily, list(daily.keys())[0])
TimeSeries(dates, counts)

**CHALLENGE 5**

In [ ]:

def perSCPs(dictionary):
    scpDict = defaultdict()
    
    for k, lol in dictionary.items():
        newK = tuple(k[:2] + (k[3],))
        try:
            scpDict[newK] += lol
        except KeyError:
            # if there is no entry for this key, make one
            scpDict[newK] = lol
        
    scpDict = perDay(scpDict)
    
    return scpDict

SCPs = perSCPs(daily)


**CHALLENGE 6**

In [ ]:
def perStations(dictionary):
    staDict = defaultdict()
    
    for k, lol in dictionary.items():
        newK = k[2]
        try:
            staDict[newK] += lol
        except KeyError:
            # if there is no entry for this key, make one
            staDict[newK] = lol
        
    staDict = perDay(staDict)
    
    return staDict

stations = perStations(SCPs)

**CHALLENGE 7**


In [ ]:
dates, counts = PlotbyDay(stations, list(stations.keys())[0])
TimeSeries(dates, counts)

**CHALLENGE 8**

In [ ]:
def SplitWeekly(d, station):
    Weeklylist = []
    weekEntries = []
    
    for i, dataList in enumerate(sorted(d[station])):
        date = dataList[0]
        weekday = date.weekday()
        entries = dataList[1]
        
        if (i == 0) and (weekday != 6):  # If first item in list and weekday is NOT Sunday
            daysAhead = (7 + weekday) - 6
            for i in range(daysAhead):
                weekEntries.append(0)
        
        weekEntries.append(entries)
        
        if (weekday == 5):  # If NOT Sunday
            Weeklylist.append(weekEntries)
            weekEntries = []
        
    lastWeekLen = len(weekEntries)
    if lastWeekLen != 0:
        for i in range(7 - lastWeekLen):
            weekEntries.append(0)

    Weeklylist.append(weekEntries)
    
    return Weeklylist

weekls = SplitWeekly(stations, list(stations.keys())[0])

In [ ]:
def WeekData(oneweek):
    # weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    weekdays = [x + 1 for x in range(7)]
    counts = oneweek
    return weekdays, counts

In [ ]:
def RainbowWeek(weeklyLists):
    for week in weeklyLists:
        weekdays, counts = WeekData(week)
        plt.plot(weekdays, counts)
        
RainbowWeek(weekls)

**CHALLENGE 9**

In [ ]:
def AllDations(d):
    return {k: sum([lst[1] for lst in LoLs]) for k, LoLs in d.items()}

alldatestation = sorted(AllDations(stations).items(), key=lambda x: x[1], reverse=1)


In [ ]:
stationCount = len(alldatestation)
print alldatestation

**CHALLENGE 10**

In [ ]:
counts = [count for station, count in alldatestation]
plt.hist(counts)

In [ ]:
indices = range(1, len(counts) + 1)
plt.bar(indices, counts)